#  CNN (konvolucione neuralne mreže)

**"A class of deep neural networks most commonly applied to analyzing visual imagery"**



## Pregled
* CNN
* Slojevi
* Novi pogled na protok podataka
* Arhitektura
* Konvolucija
* Pooling
* Arhitektura - ponovo
* Česti problemi

## Konvolucione neuralne mreže (ConvNet, CNN)
* Tip NN specijalno namenjen slučajevima kada su ulazni podaci slike
  * Opštije: kada je umesto feature vektora ulaz predstavljen matricom pri čemu postoje pravilnosti u obe dimenzije
* **Kao i MLP**: feedforward mreža, sastoji se od neurona sa aktivacionim funkcijama, može imati raznovrsnu arhitekturu, na isti način se vrši forward propagation, iste su funkcije troška, na isti način se vrši trening (backpropagation od troška po slojevima unazad)
* **Za razliku od MLP**: novi tipovi slojeva pogodni za rad sa slikama, novi pogled na protok podataka, standardni šabloni po pitanju arhitekture

## Slojevi
* U slučaju MLP imali smo samo jedan tip sloja: **potpuno povezan** (Fully Connected, FC, Dense)
  * Ovaj sloj je definisan trojkom  $(W, b, f)$ - matrica težina, bias vektor, funkcija aktivacije
  * Ulaz (vektor) se množi sa $W$, dodaje $b$ i zatim primenjuje $f$ kako bi se dobio novi vektor
* U slučaju CNN uvodimo dva nova, malo drukčija, sloja: **konvoluciju** (Conv) i **pooling** (Pool)
* Takođe, aktivacione funkcije možemo zarad jednostavnosti posmatrati kao slojeve za sebe

## Novi pogled na protok podataka
* CNN svaki međurezultat posmatraju (umesto kao feature vektor) kao 3D tenzor $(W,H,C)$ (širina, visina, broj kanala tj. dubina)
* Ulazni podaci su slika tj. 3D tenzor npr. $(W, H, 3)$ u slučaju RGB ili $(W, H, 1)$ u slučaju BW slika
* [Ilustracija novog pogleda na protok podataka](http://cs231n.github.io/assets/cnn/cnn.jpeg)
  
## Arhitektura
* Jedna CNN će se sastojati od niza **Conv** i **Pool** slojeva koji transformišu 3D tenzor (tj. sliku) u novi 3D tenzor (tj. novu "sliku", iako dobijene "slike" brzo prestaju da imaju smisla vizuelno - barem ne na onaj način na koji bismo očekivali)
* Na samom kraju ćemo uglavnom imati jedan ili više FC slojeva koji, kao i inače, rade nad vektorskom reprezentacijom
* **Dakle**: Conv i Pool transformišu sliku tj. skup feature-a, a finalni FC slojevi (tj. finalni MLP) koristi te feature-e umesto originalnih kao vektor nad kojim vrši predikciju
* [Ilustracija arhitekture 1](http://cs231n.github.io/assets/cnn/convnet.jpeg)
* [Ilustracija arhitekture 2](https://cdn-images-1.medium.com/max/1600/1*uAeANQIOQPqWZnnuH-VEyw.jpeg)

## Konvolucija
* **Motivacija**: Primena FC sloja na RGB sliku 200x200 zahteva 120000 težina po neuronu. Ovaj broj je veliki jer svaki neuron "posmatra" svaki drugi
* Treba nam sloj koji bolje koristi prostorne pravilnosti u slici a ima manje parametara od FC
* **Ideja**: Svaki neuron posmatra samo deo ulaza umesto ceo ulaz (sada je jasno zašto se FC zove FC tj. Dense)
* Podsetnik: ulaz je 3D tenzor $(W, H, C)$
* Konvolucioni sloj se sastoji od niza kernela (filtera)
* Jedan kernel je 3D tenzor $(K, K, C)$
* Svaki kernel proizvodi jedan kanal izlazne slike ($N_K$ kernela daje izlazni 3D tenzor dimenzija $(W', H', N_K)$, gde se $W'$ i $H'$ mogu lako izračunati)
* Sada se fokusirajmo na jedan kernel $(K, K, C)$ i kako on vrši transformaciju $(W,H,C) \to (W', H')$
* Ovde dolazimo do operacije **konvolucije**: prevlačimo kernel preko ulazne slike po širini i dužini (kernel je uvek iste dubine tj. broja kanala kao ulaz) i za svaku poziciju računamo skalarni proizvod vrednosti u ulazu i u kernelu i to upisujemo kao jednu od vrednosti na izlazu
* Posmatrajmo za početak slučaj C=1
* [Prikaz konvolucije za C=1 i K=3](https://cdn-images-1.medium.com/max/1200/1*1VJDP6qDY9-ExTuQVEOlVg.gif)
* [Prikaz konvolucije za C=1 i K=5](https://cdn-images-1.medium.com/max/1200/1*nYf_cUIHFEWU1JXGwnz-Ig.gif)
* [Prikaz konvolucije sa konkretnim vrednostima za C=1 i K=3](https://cdn-images-1.medium.com/max/1200/1*uoWYsCV5vBU8SHFPAPao-w.gif)
* Animacija na [cs231n beleškama za CNN](http://cs231n.github.io/convolutional-networks/) najbolje ilustruje konvolucioni sloj i prikazuje slučja kada imamo više kanala i više kernela
* Sada kada smo razumeli konvoluciju možemo pokušati da je posmatramo na način na koji smo definisali MLP: koristeći pojam neurona
  * U kontekstu neurona jedan kernel se može posmatrati kao $W' * H'$ neurona pri čemu svaki ima svoje **receptivno polje** (dakle nije "dense")
  * Svi neuroni (po dubini) koji su na istoj poziciji u okviru kernela imaju isto receptivno polje
  * Svi neuroni koji su deo istog kernela dele težine
  * Ako su bitne horizontalne ivice, bitne su svuda u okviru slike, pa ima smisla da koristimo iste težine za različita receptivna polja
  * Deljenje težina dovodi do bolje efikasnosti Conv slojeva
  * [Prikaz pogleda na CNN u kontekstu neurona](http://cs231n.github.io/assets/cnn/depthcol.jpeg)
* **Parametri Conv sloja**: broj kernela, veličina kernela, stride, padding (isti za jedan Conv sloj)
* **Rezultat**: jednostavni feature-i na nižim, a kompleksniji na višim slojevima

## Pooling
- Za razliku od Conv, Pool je dosta jednostavniji sloj **bez learnable parametara** 
- Ima dva fiksna parametra, dimenziju i stride
- Služi da smanji $W$ i $H$ (pritom ne menja broj kanala)
- Svaki Pool sloj deli ulaznu sliku (po širini i dužini) na kvadrate fiksne veličine (često 2 ili 3) i "sumira" ih nekom funkcijom (max, min, avg)
- [Ilustracija](http://cs231n.github.io/assets/cnn/pool.jpeg)

## Arhitektura - ponovo
- Najčešće se koristi arhitektura nalik sledećoj:

```
 [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]*K -> FC
```
- **ILSVRC**: takmičenje u klasifikaciji slika na ImageNet skupu (1000 kategorija, 1.2M slika)
  - Često se koristi kao merilo "najbolje" konvolucione mreže
  - **Skorašnji pobednici**: AlexNet, LeNet, GoogLeNet, VGGNet, ResNet
- **Transfer learning**: mrežu treniranu za jedan zadatak primenjujemo na drugi
  - Tako možemo konvolucioni deo ResNet-a pretreniranog na ImageNet (koji sami ne bismo nikad uspeli da istreniramo) da primenimo na bilo koji problem kao feature extraction korak (i samo dodamo MLP na kraj koji "dotreniramo" za konkretan zadatak

## Česti problemi
- [Odličan pregled aktuelnih problema i radova - na dnu članka](https://skymind.ai/wiki/convolutional-network)

## Resursi
- [cs231n: Detaljan uvod u CNN, dobro za razvijanje intuicije](http://cs231n.github.io/convolutional-networks/)
- [Kraći uvod, dobre animacije](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53)

## Implementacija CNN u Keras
* U TF bi takođe bilo relativno jednostavno (`tf.layers.conv2d` i `tf.layers.max_pooling2d`) ali sada nema razloga da ne koristimo Keras (fokus je na višem nivou)
* Skup [CIFAR-10](https://skymind.ai/wiki/convolutional-network) za klasifikaciju slika ("lakša verzija ImageNet")
* 60000 32x32 RGB slika u 10 klasa (svaka klasa ima jednako slika), 5/6 toga je trening test

In [0]:
'''
Treniramo jednostavan duboki CNN na CIFAR10
Dobijamo 75% accuracy na validacionom skupu za 25 epoha, što nije ni blizu 
dovoljno tj. mreža bi nastavila da uči i nakon toga, a trening već traje dugo
'''

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# Parametri
batch_size = 32
num_classes = 10
epochs = 25
num_predictions = 20

# Vrsimo "data augmentation": prosirujemo skup podataka jednostavnim metodama
# obrade slike kako bismo dobili vise materijala za treniranje
data_augmentation = True

# Ucitavamo CIFAR10 trening i test podatke
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)

# Kreiramo one-hot vektore
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)

# Kreiramo model
# Prvi sloj je Conv sloj sa 32 kernela 3x3 i 'same' strategijom paddinga
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # Max pooling 2x2
model.add(Dropout(rate=0.75))  # Dropout metoda regularizacije

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.75))

model.add(Flatten())
model.add(Dense(512))  # Prvi potpuno povezan sloj
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(num_classes))  # Finalni potpuno povezan sloj
model.add(Activation('softmax'))

# Koristimo RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Kompilacija modela
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Normalizacija podataka
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    # Prosto treniranje modela bez augmentacije
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # Preprocesiranje augmentovanih podataka
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Neke vrednosti potrebne za augmentaciju podataka je neophodno fitovati
    datagen.fit(x_train)

    # Fitujemo model na augmentovanim podacima sa 4 workera
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        workers=4)

# Ispisujemo finalni rezultat na test skupu
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])